# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [31]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np
from sklearn.linear_model import LogisticRegression


In [32]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [33]:
# frequency table for prestige and whether or not someone was admitted
#admit_prestige = pd.crosstab(index=df["admit"], 
#                           columns=df["prestige"])

#admit_prestige.index= ["admit","reject"]
# why couldn't i get the other way to work?  
pd.crosstab(df.admit, df.prestige)

prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [34]:
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')


#### 2.2 When modeling our class variables, how many do we need? 



Answer:  We need four dummy variables because there are 4 levels of prestigue. 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [35]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':]) 
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             0             1             0
1      1  660.0  3.67             0             0             1             0
2      1  800.0  4.00             1             0             0             0
3      1  640.0  3.19             0             0             0             1
4      0  520.0  2.93             0             0             0             1


/Users/annegriffin/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  


In [36]:
df.columns.tolist()

['admit', 'gre', 'gpa', 'prestige']

In [37]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
# are they asking me to cut admission and prestige 1?  or to only look at them?
# cols_to_keep = 
pd.crosstab(handCalc["admit"], handCalc['prestige_1.0']) #i don't know why this doesn't work

prestige_1.0    0   1
admit                
0             243  28
1              93  33

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

33/61

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

31/112

#### 3.3 Calculate the odds ratio

8019/2604 = 3.08

#### 3.4 Write this finding in a sentenance: 

Answer: The odds of being addmited to a grad school if you attended a #1 ranked college is 3.08 higher than if you went to a school that was not #1 ranked.

#### 3.5 Print the cross tab for prestige_4

In [38]:
pd.crosstab(handCalc["admit"], handCalc['prestige_4.0'])

prestige_4.0    0   1
admit                
0             216  55
1             114  12

#### 3.6 Calculate the OR 

2592/6270 = 0.413

#### 3.7 Write this finding in a sentence

Answer: If you attended a #4 ranked school, your odds of getting admited to grad school 0.413 lower than if you attended a school that was ranked higher.

## Part 4. Analysis

In [39]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61             0             1             0
1      1  660.0  3.67             0             1             0
2      1  800.0  4.00             0             0             0
3      1  640.0  3.19             0             0             1
4      0  520.0  2.93             0             0             1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [40]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [105]:
train_cols = ['gre' , 'gpa']

#### 4.2 Fit the model

In [106]:
#use sm
lm = LogisticRegression()
label = 'admit'
lm.fit(data.loc[:, train_cols], data[label])

#logit = sm.Logit(data['admit], data[train_cols])
#result = logit.fit()

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [107]:
result = sm.Logit(data["admit"], data[train_cols]).fit()

Optimization terminated successfully.
         Current function value: 0.629250
         Iterations 5


#### 4.3 Print the summary results

In [49]:
print 'Coefficients:', lm.coef_
print 'Intercept:', lm.intercept_
print 'Bias:', df.admit.mean()
print 'Accuracy', lm.score(data.loc[:, train_cols], data[label])

Coefficients: [[  6.52246628e+00  -1.09140320e-04  -6.30332513e-01  -6.93042050e-01]]
Intercept: [-0.69304205]
Bias: 0.317380352645
Accuracy 1.0


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [77]:
or_column = np.exp(lm.coef_)
print or_column

[[  6.80254018e+02   9.99890866e-01   5.32414737e-01   5.00052568e-01]]


In [78]:
#data['OR'] = or_column
#df.assign(OR = np.exp(lm.coef_))

In [79]:
# calculate z score = likelihood indicator use that to calc CI
#conf.columns = ['2.5%', '97.5%', 'OR']

#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [62]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out
# cartesian product is every outcome

In [109]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [110]:
combos.head()

0         1    2    3
0  220.0  2.260000  1.0  1.0
1  220.0  2.260000  2.0  1.0
2  220.0  2.260000  3.0  1.0
3  220.0  2.260000  4.0  1.0
4  220.0  2.453333  1.0  1.0

#### 5.1 Recreate the dummy variables

In [112]:
newcols = [0,1, 2, 3]
data2 = combos[newcols].join(dummy_ranks.ix[:, "prestige_2.0":])
data2.head()

0         1    2    3  prestige_2.0  prestige_3.0  prestige_4.0
0  220.0  2.260000  1.0  1.0             0             0             0
1  220.0  2.260000  2.0  1.0             1             0             0
2  220.0  2.260000  3.0  1.0             0             1             0
3  220.0  2.260000  4.0  1.0             0             0             1
4  220.0  2.453333  1.0  1.0             0             0             0

In [98]:
# recreate the dummy variables
#cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])

ValueError: Length mismatch: Expected axis has 7 elements, new values have 4 elements

#### 5.2 Make predictions on the enumerated dataset

In [124]:
data2.dropna()

0         1    2    3  prestige_2.0  prestige_3.0  prestige_4.0
0    220.0  2.260000  1.0  1.0             0             0             0
1    220.0  2.260000  2.0  1.0             1             0             0
2    220.0  2.260000  3.0  1.0             0             1             0
3    220.0  2.260000  4.0  1.0             0             0             1
4    220.0  2.453333  1.0  1.0             0             0             0
5    220.0  2.453333  2.0  1.0             1             0             0
6    220.0  2.453333  3.0  1.0             0             1             0
7    220.0  2.453333  4.0  1.0             0             0             1
8    220.0  2.646667  1.0  1.0             0             0             0
9    220.0  2.646667  2.0  1.0             1             0             0
10   220.0  2.646667  3.0  1.0             0             1             0
11   220.0  2.646667  4.0  1.0             0             0             1
12   220.0  2.840000  1.0  1.0             0             0             0
13   220.0  2.840000  2.0  1.0             1             0             0
14   220.0  2.840000  3.0  1.0             0             1             0
15   220.0  2.840000  4.0  1.0             0             0             1
16   220.0  3.033333  1.0  1.0             0             0             0
17   220.0  3.033333  2.0  1.0             1             0             0
18   220.0  3.033333  3.0  1.0             0             1             0
19   220.0  3.033333  4.0  1.0             0             0             1
20   220.0  3.226667  1.0  1.0             0             0             0
21   220.0  3.226667  2.0  1.0             1             0             0
22   220.0  3.226667  3.0  1.0             0             1             0
23   220.0  3.226667  4.0  1.0             0             0             1
24   220.0  3.420000  1.0  1.0             0             0             0
25   220.0  3.420000  2.0  1.0             1             0             0
26   220.0  3.420000  3.0  1.0             0             1             0
27   220.0  3.420000  4.0  1.0             0             0             1
28   220.0  3.613333  1.0  1.0             0             0             0
29   220.0  3.613333  2.0  1.0             1             0             0
..     ...       ...  ...  ...           ...           ...           ...
370  800.0  2.646667  3.0  1.0             0             1             0
371  800.0  2.646667  4.0  1.0             0             0             1
372  800.0  2.840000  1.0  1.0             0             0             0
373  800.0  2.840000  2.0  1.0             1             0             0
374  800.0  2.840000  3.0  1.0             0             1             0
375  800.0  2.840000  4.0  1.0             0             0             1
376  800.0  3.033333  1.0  1.0             0             0             0
377  800.0  3.033333  2.0  1.0             1             0             0
378  800.0  3.033333  3.0  1.0             0             1             0
379  800.0  3.033333  4.0  1.0             0             0             1
380  800.0  3.226667  1.0  1.0             0             0             0
381  800.0  3.226667  2.0  1.0             1             0             0
382  800.0  3.226667  3.0  1.0             0             1             0
383  800.0  3.226667  4.0  1.0             0             0             1
384  800.0  3.420000  1.0  1.0             0             0             0
385  800.0  3.420000  2.0  1.0             1             0             0
386  800.0  3.420000  3.0  1.0             0             1             0
387  800.0  3.420000  4.0  1.0             0             0             1
388  800.0  3.613333  1.0  1.0             0             0             0
389  800.0  3.613333  2.0  1.0             1             0             0
390  800.0  3.613333  3.0  1.0             0             1             0
391  800.0  3.613333  4.0  1.0             0             0             1
392  800.0  3.806667  1.0  1.0             0             0         

In [126]:
predict_columns = data2[[0,1, "prestige_2.0"]] #add variables from the new data frameres
result2 = lm.predict(predict_columns)

ValueError: X has 3 features per sample; expecting 2

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.